In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

from sklearn.model_selection import train_test_split

In [2]:
torch.manual_seed(0)
np.random.seed(0)

In [3]:
df = pd.read_csv("/Users/michaelhein/Programs/Dicebot/outputs/output.txt")

In [4]:
df.head()

totalDice     player  numPlayerDice  quantityDeclared  diceNumDeclared  1s  \
0         25   Agg-High              5                 9                4   2   
1         24    Agg-Low              5                 8                4   0   
2         24   Cond-Low              5                 9                2   1   
3         23  Cond-High              5                 9                3   1   
4         22    Conserv              4                 1                4   0   

   2s  3s  4s  5s  6s  
0   0   0   3   0   0  
1   0   1   2   2   0  
2   2   1   0   0   1  
3   0   3   0   1   0  
4   1   0   2   0   1

In [5]:
df["totalDice"]

0         25
1         24
2         24
3         23
4         22
          ..
590880     6
590881     6
590882     5
590883     4
590884     4
Name: totalDice, Length: 590885, dtype: int64

In [6]:
df.shape

(590885, 11)

In [7]:
df[["totalDice", "player", "numPlayerDice", "quantityDeclared", "diceNumDeclared"]]

totalDice     player  numPlayerDice  quantityDeclared  diceNumDeclared
0              25   Agg-High              5                 9                4
1              24    Agg-Low              5                 8                4
2              24   Cond-Low              5                 9                2
3              23  Cond-High              5                 9                3
4              22    Conserv              4                 1                4
...           ...        ...            ...               ...              ...
590880          6   Agg-High              1                 2                5
590881          6    Agg-Low              1                 3                4
590882          5  Cond-High              3                 2                3
590883          4    Agg-Low              1                 1                4
590884          4  Cond-High              3                 2                1

[590885 rows x 5 columns]

In [8]:
features = df[df.columns[:5]]
features.head()

totalDice     player  numPlayerDice  quantityDeclared  diceNumDeclared
0         25   Agg-High              5                 9                4
1         24    Agg-Low              5                 8                4
2         24   Cond-Low              5                 9                2
3         23  Cond-High              5                 9                3
4         22    Conserv              4                 1                4

In [9]:
targets = df[df.columns[5:]]
targets.head()

1s  2s  3s  4s  5s  6s
0   2   0   0   3   0   0
1   0   0   1   2   2   0
2   1   2   1   0   0   1
3   1   0   3   0   1   0
4   0   1   0   2   0   1

In [10]:
Conserv = df[df["player"] == "Conserv"]
Conserv = Conserv.drop(columns = ["player"])
Conserv.head()

totalDice  numPlayerDice  quantityDeclared  diceNumDeclared  1s  2s  3s  \
4          22              4                 1                4   0   1   0   
9          21              3                 1                1   2   1   0   
14         19              2                 1                3   0   0   1   
19         18              1                 1                5   0   0   0   
58         24              4                 1                2   0   1   1   

    4s  5s  6s  
4    2   0   1  
9    0   0   0  
14   0   0   1  
19   0   1   0  
58   0   0   2

In [11]:
td, test = train_test_split(Conserv, test_size=0.1)

In [12]:
train, develop = train_test_split(Conserv, test_size=0.2)

In [13]:
train.shape

(95698, 10)

In [14]:
develop.shape

(23925, 10)

In [15]:
test.shape

(11963, 10)

In [16]:
x_train = train[train.columns[:4]]
y_train = train[train.columns[4:]]

x_develop = develop[develop.columns[:4]]
y_develop = develop[develop.columns[4:]]

x_test = test[test.columns[:4]]
y_test = test[test.columns[4:]]

In [17]:
x_train_tensor = torch.from_numpy(x_train.to_numpy()).float()
y_train_tensor = torch.from_numpy(y_train.to_numpy()).float()

x_develop_tensor = torch.from_numpy(x_develop.to_numpy()).float()
y_develop_tensor = torch.from_numpy(y_develop.to_numpy()).float()

x_test_tensor = torch.from_numpy(x_test.to_numpy()).float()
y_test_tensor = torch.from_numpy(y_test.to_numpy()).float()

In [18]:
train_dataset = list(zip(x_train_tensor, y_train_tensor))
develop_dataset = list(zip(x_develop_tensor, y_develop_tensor))
test_dataset = list(zip(x_test_tensor, y_test_tensor))

In [19]:
class Net(nn.Module):
    def __init__(self, num_layers, layers_size):
        super(Net, self).__init__()
        self.hidden_layers = nn.ModuleList([nn.Linear(4, layers_size)])
        self.hidden_layers.extend([nn.Linear(layers_size, layers_size) for i in range(1, num_layers-1)])
        self.output = nn.Linear(layers_size, 6)
        self.activation = nn.ReLU()
        
    def forward(self, x):
        z = x
        for layer in self.hidden_layers:
            z = self.activation(layer(z))
        return self.output(z)

In [20]:
class Trainer():
    def __init__(self, net, optim, loss_function, train_loader):
        self.net = net
        self.optim = optim
        self.loss_function = loss_function
        self.train_loader = train_loader
        
    def train(self, epochs, verbose=True):
        losses = []
        for epoch in range(epochs):
            epoch_loss = 0.0
            epoch_steps = 0
            for data in self.train_loader:
                x = data[0]
                y = data[1]
                
                self.optim.zero_grad()
                
                preds = self.net.forward(x)
                
                loss = self.loss_function(preds, y)
                
                loss.backward()
                
                self.optim.step()
                
                epoch_loss += loss.item()
                epoch_steps += 1
                
            losses.append(epoch_loss / epoch_steps)
            if verbose: print("epoch [%d]: loss %.3f" % (epoch+1, losses[-1]))
        return losses

In [21]:
## Hyperparamters

num_epochs = 30
batch_size = 256
learning_rate = 0.02

In [22]:
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
develop_loader = DataLoader(dataset=develop_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

In [23]:
net = Net(2, 16)
opt = optim.Adam(net.parameters(), lr=learning_rate)
loss_function = nn.MSELoss()

trainer = Trainer(net=net, optim=opt, loss_function=loss_function, train_loader=train_loader)

losses = trainer.train(num_epochs)

epoch [1]: loss 0.386
epoch [2]: loss 0.335
epoch [3]: loss 0.308
epoch [4]: loss 0.303
epoch [5]: loss 0.294
epoch [6]: loss 0.287
epoch [7]: loss 0.280
epoch [8]: loss 0.279
epoch [9]: loss 0.276
epoch [10]: loss 0.275
epoch [11]: loss 0.274
epoch [12]: loss 0.274
epoch [13]: loss 0.273
epoch [14]: loss 0.273
epoch [15]: loss 0.272
epoch [16]: loss 0.274
epoch [17]: loss 0.273
epoch [18]: loss 0.274
epoch [19]: loss 0.273
epoch [20]: loss 0.272
epoch [21]: loss 0.274
epoch [22]: loss 0.274
epoch [23]: loss 0.272
epoch [24]: loss 0.273
epoch [25]: loss 0.272
epoch [26]: loss 0.273
epoch [27]: loss 0.273
epoch [28]: loss 0.273
epoch [29]: loss 0.272
epoch [30]: loss 0.273


In [24]:
Conserv.head()

totalDice  numPlayerDice  quantityDeclared  diceNumDeclared  1s  2s  3s  \
4          22              4                 1                4   0   1   0   
9          21              3                 1                1   2   1   0   
14         19              2                 1                3   0   0   1   
19         18              1                 1                5   0   0   0   
58         24              4                 1                2   0   1   1   

    4s  5s  6s  
4    2   0   1  
9    0   0   0  
14   0   0   1  
19   0   1   0  
58   0   0   2

In [25]:
train_dataset[0]

(tensor([21.,  3.,  1.,  5.]), tensor([0., 0., 0., 0., 2., 1.]))

In [26]:
train_dataset[1]

(tensor([18.,  2.,  1.,  5.]), tensor([0., 0., 0., 0., 2., 0.]))

In [27]:
x_train.head()

totalDice  numPlayerDice  quantityDeclared  diceNumDeclared
217105         21              3                 1                5
405771         18              2                 1                5
23420           9              2                 3                4
551974          6              1                 2                5
452403         13              4                 2                1

In [28]:
net.forward(train_dataset[0][0])

tensor([0.3085, 0.2476, 0.3061, 0.5035, 0.9996, 0.5610],
       grad_fn=<AddBackward0>)

In [29]:
net.forward(train_dataset[1][0])

tensor([0.1472, 0.0800, 0.1532, 0.3550, 0.8056, 0.3915],
       grad_fn=<AddBackward0>)

In [30]:
develop_dataset[0]

(tensor([22.,  5.,  7.,  5.]), tensor([1., 0., 1., 1., 1., 1.]))

In [31]:
develop_dataset[1]

(tensor([5., 1., 2., 2.]), tensor([0., 0., 0., 0., 0., 1.]))

In [32]:
develop_dataset[2]

(tensor([21.,  4.,  1.,  4.]), tensor([0., 1., 1., 1., 1., 0.]))

In [33]:
develop_dataset[3]

(tensor([18.,  4.,  7.,  2.]), tensor([0., 2., 0., 0., 0., 2.]))

In [34]:
net.forward(develop_dataset[0][0])

tensor([0.6304, 0.5374, 0.7061, 0.7600, 1.3798, 0.9022],
       grad_fn=<AddBackward0>)

In [35]:
net.forward(develop_dataset[1][0])

tensor([-0.0613,  1.1027, -0.0804,  0.0744, -0.1498,  0.1430],
       grad_fn=<AddBackward0>)

In [36]:
net.forward(develop_dataset[2][0])

tensor([0.4577, 0.4008, 0.4852, 1.5046, 0.5170, 0.5534],
       grad_fn=<AddBackward0>)

In [37]:
net.forward(develop_dataset[3][0])

tensor([0.4069, 1.5966, 0.3370, 0.5770, 0.3735, 0.6226],
       grad_fn=<AddBackward0>)

In [38]:
torch.save(net.state_dict(), "/Users/michaelhein/Programs/Dicebot/models/conservative_stat.obj")